- Read energy_unique_text.csv (from Step 2 output)
- Remove LongDescription with single words
- Remove French parts inside the LongDescription using regex
- Detect language for each cleaned LongDescription
- Translate non-English text to English (if needed)
- Save final output as energy_text_trn.csv

model used for detencting language: https://huggingface.co/facebook/fasttext-language-identification

The translation step uses Google Translate API.

In [ ]:
# Install dependencies
!pip install -U fasttext
!pip install -U googletrans==4.0.0-rc1
!pip install -U huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313503 sha256=d81ca289d2626a04e93c6b29d7a865e4c90975a017c7b491cad5fa2d4c1363d4
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.7 MB/s eta 0:00:00
   ━━━

In [ ]:
#FIX: Downgrade NumPy to 1.x
!pip install numpy==1.24.4 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 122.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.1 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is

In [ ]:
!pip install langdetect
!pip install deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=fdbf930f15d7678b70c5db219aef1dcb2a1a5edeee67a60ab562fc9f6028d54f
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import fasttext
from langdetect import detect
#from deep_translator import GoogleTranslator
from tqdm import tqdm
from huggingface_hub import hf_hub_download
from googletrans import Translator

import os

In [ ]:
tqdm.pandas()  # enable progress_apply()

## Load the dataset

In [ ]:
# Load dataset
df = pd.read_csv('/content/energy_unique_text.csv')
print(f"Loaded dataset with {len(df)} records.")

Loaded dataset with 79324 records.


## Remove LD with single words

In [ ]:
# Keep only LongDescriptions with at least two words
df = df[df['LongDescription'].astype(str).str.split().str.len() >= 2]

print(f"df {len(df)} records.")

df 64212 records.


## Remove French Part

In [ ]:
# Compile a regex pattern for French intro phrases

french_start_regex = re.compile(
    r"/\s*(Le projet|Le Partenariat|Cette subvention|Ce projet|Le CIGI est|Le fonds|Cette initiative|Le programme|Avec un|Le but ultime|Les changements|Les investissements|ConÃ§u en |Conçu|Conçu en|L'initiative|Au Sommet|Les progr|Le Développement|La Facilit| Certaines des innovations)",
    re.IGNORECASE
)

In [ ]:
def remove_french_translation(text):
    match = french_start_regex.search(text)
    if match:
        return text[:match.start()].strip()  # keep everything before the match
    return text

In [ ]:
df['LD_Eng'] = df['LongDescription'].astype(str).apply(remove_french_translation)

In [ ]:
df['FrenchPartRemoved'] = df['LongDescription'] != df['LD_Eng']
df['FrenchPartRemoved'].value_counts()

,count
FrenchPartRemoved,
False,63861
True,351


In [ ]:
df[df['FrenchPartRemoved'] == True][['LongDescription', 'LD_Eng']].sample(5, random_state=42)

,LongDescription,LD_Eng
43577,This project aims to improve the livelihoods o...,This project aims to improve the livelihoods o...
73854,The project will monitor and assess the Intern...,The project will monitor and assess the Intern...
72914,"At the G7 Summit in May 2023, Canada announced...","At the G7 Summit in May 2023, Canada announced..."
67020,This project contributes to a five-year Exper...,This project contributes to a five-year Exper...
43575,This initiative is the second phase of the Can...,This initiative is the second phase of the Can...


## Detect Language

In [ ]:
# Load the fasttext model
model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
lang_model = fasttext.load_model(model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.bin:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

In [ ]:
# Define helper functions
def detect_language(text):
    try:
        label, prob = lang_model.predict(str(text))
        lang = label[0].replace("__label__", "")
        return lang
    except:
        return "unknown"

translator = Translator()

def translate_if_needed(text, lang_code):
    if lang_code != "eng_Latn":
        try:
            return translator.translate(text, dest='en').text
        except:
            return text
    return text

In [ ]:
# Language detection
df['DetectedLanguage'] = df['LongDescription'].progress_apply(detect_language)

100%|██████████| 64212/64212 [00:13<00:00, 4733.94it/s]


In [ ]:
# Language summary
df['DetectedLanguage'].value_counts().reset_index().head(5)

,DetectedLanguage,count
0,eng_Latn,47007
1,yue_Hant,9139
2,fra_Latn,3588
3,kor_Hang,2375
4,deu_Latn,714


In [ ]:
# Translation
df['TranslatedDescription'] = df.progress_apply(lambda row: translate_if_needed(row['LongDescription'], row['DetectedLanguage']), axis=1)

100%|██████████| 64212/64212 [1:24:28<00:00, 12.67it/s] 


In [ ]:
df.to_csv('/content/energy_text_trn.csv', index=False)